## Initialization

In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [2]:
# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# install findspark using pip
!pip install -q findspark

In [3]:
pip install koalas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.2/593.2 kB 9.7 MB/s eta 0:00:00


In [4]:
pip install --upgrade koalas

In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


***


## Functions

In [27]:
!ls /content/drive/MyDrive/Spark/stocks_data

AMAZON.csv  APPLE.csv  FACEBOOK.csv  GOOGLE.csv  MICROSOFT.csv	TESLA.csv  ZOOM.csv


In [8]:
from warnings import filters

def read_file(folder, file):
  name = folder + file
  df = spark.read.csv(name,inferSchema=True, header =True)
  return df

In [9]:
from pyspark.sql.functions import col, to_date

def calculate_time_span(dataset):
  dataset2 = dataset.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))
  min = dataset2.agg({"Date": "min"}).collect()[0][0]
  max = dataset2.agg({"Date": "max"}).collect()[0][0]

  day_difference = max - min

  return min, max, day_difference

In [10]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
import numpy as np

def calcultate_corr(dataset):
  # convert to vector
  vector_col = "correlations"
  numeric = dataset.drop(*["Date", "company_name"])

  assembler = VectorAssembler(inputCols=numeric.columns, outputCol=vector_col)
  output = assembler.transform(numeric)

  matrix = Correlation.corr(output, vector_col)
  matrix2 = matrix.collect()[0]['pearson({})'.format(vector_col)].values

  num_columns = len(numeric.columns)
  correlation_matrix = np.array(matrix2).reshape((num_columns, num_columns))

  return correlation_matrix

In [11]:
def get_info_df(dataset):
  print(f"Infos of the {file} file \n\n")

  print("► Schema of the dataset")
  print(dataset.printSchema())

  print("\n ► Top rows of the dataset")
  print(dataset.show(10))

  print("\n ► Row count")
  print(dataset.count())

  print("\n ► Time span")
  min_date, max_date, span = calculate_time_span(dataset)
  print(f"Minimum date {min_date}")
  print((f"Maximum date {max_date}"))
  print(span)

  print("\n ► Columns info")
  for name, col in zip(dataset.schema.names, dataset.columns):
    print(name)
    print(dataset.describe([col]).show())

  print("\n ► Missing values for each column")
  for col in dataset.columns:
    print(col, "\t", "number of null values: ", dataset.filter(dataset[col].isNull()).count())

  print("\n ► Correlation Matrix")
  correlation = calcultate_corr(dataset)
  print(correlation)


In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, avg

def get_average_year(dataset, column):
  poulet = dataset.withColumn("Date", col("Date").cast("Date"))

  toAverage = poulet.groupBy(year("date").alias("year")).agg(avg(column).alias(f"average_{column}"))
  toAverage = toAverage.orderBy(year("date"))

  return toAverage

In [13]:
from pyspark.sql.functions import col, year, month, avg

def get_average_month(dataset, column):
  poulet = dataset.withColumn("Date", col("Date").cast("Date"))

  toAverage = poulet.groupBy(year("date").alias("year"), month("date").alias("month")).agg(avg(column).alias(f"average_{column}"))
  toAverage = toAverage.orderBy(year("date"), month("date"))

  return toAverage

In [14]:
from pyspark.sql.functions import col, year, weekofyear, avg

def get_average_week(dataset, column):
  poulet = dataset.withColumn("Date", col("Date").cast("Date"))

  toAverage = poulet.groupBy(year("date").alias("year"), weekofyear("date").alias("week")).agg(avg(column).alias(f"average_{column}"))
  toAverage = toAverage.orderBy(year("date"), weekofyear("date"))

  return toAverage

In [15]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F

def daily_difference(dataset):
  windowSpec = Window().orderBy("date")

  dataset_day_diff = dataset.withColumn("stock_difference", F.col("Close") - F.lag("Close").over(windowSpec))

  return dataset_day_diff

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def monthly_difference(dataset):
  window_spec = Window.partitionBy(F.year("date"), F.month("date")).orderBy("date")

  df_with_row_number = dataset.withColumn("row_number", F.row_number().over(window_spec))
  result_df = df_with_row_number.filter(F.col("row_number") == 1).drop("row_number")

  windowSpec2 = Window().orderBy("date")

  dataset_month_diff = result_df.withColumn("stock_difference", F.col("Close") - F.lag("Close").over(windowSpec2))

  return dataset_month_diff

In [17]:
def calculate_daily_return(dataset):
  newDataset = dataset.withColumn("Daily_return",  col("Close") - col("Open"))

  return newDataset

In [18]:
def get_highest_dr(dataset, start_date=None, end_date=None):
    if "Daily_return" in dataset.columns:
        if start_date is not None and end_date is not None:
            filteredDataset = dataset.filter((col("Date") >= start_date) & (col("Date") <= end_date))
            sortedAndFilteredDataset = filteredDataset.orderBy(col("Daily_return").desc())
            return sortedAndFilteredDataset
        else:
            sortedDataset = dataset.orderBy(col("Daily_return").desc())
            return sortedDataset
    else:
        print("Error: 'Daily_return' column not found.")
        return None

In [19]:
def calculate_moving_average(df, column_name, n, start_date=None):

    window_spec = Window.orderBy("date").rowsBetween(-n + 1, 0)

    moving_avg_col = f"{column_name}_moving_avg"
    df_with_moving_avg = df.withColumn(moving_avg_col, F.avg(column_name).over(window_spec))

    if start_date is not None:
        result_df = df_with_moving_avg.filter(F.col("date") >= start_date)
    else:
        result_df = df_with_moving_avg

    return result_df

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def get_ror_daytoday(dataset, reference_date, period):
    spark = SparkSession.builder.getOrCreate()

    dataset = dataset.withColumn("date", F.to_date("date"))

    end_date = reference_date + F.expr(f"INTERVAL {period} DAYS")

    filtered_data = dataset.filter((F.col("date") >= reference_date) & (F.col("date") <= end_date))

    window_spec = Window().orderBy("date")

    return_rate_column = "return_rate"
    dataset = (filtered_data
               .withColumn(return_rate_column,
                           (F.col("close") - F.lag("close").over(window_spec)) / (F.lag("close").over(window_spec))*100)
               .na.drop()
               .orderBy("date"))

    return dataset.orderBy(F.desc("return_rate"))

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def get_ror_period(dataset, reference_date, period):
    spark = SparkSession.builder.getOrCreate()

    dataset = dataset.withColumn("date", F.to_date("date"))

    end_date = reference_date + F.expr(f"INTERVAL {period} DAYS")

    filtered_data = dataset.filter((F.col("date") >= reference_date) & (F.col("date") <= end_date))

    return_rate_column = "return_rate"
    initial_close_price = filtered_data.filter(F.col("date") == reference_date).select("close").collect()[0][0]

    dataset = (filtered_data
               .withColumn(return_rate_column,
                           (F.col("close") - F.lit(initial_close_price)) / F.lit(initial_close_price))
               .orderBy("date"))

    return dataset

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

def calculate_stock_correlation(stock1, stock2, start_date=None, end_date=None):

    stock1 = stock1.select("date", "close").withColumnRenamed("close", "close1")
    stock2 = stock2.select("date", "close").withColumnRenamed("close", "close2")

    if start_date and end_date:
        stock1 = stock1.filter((F.col("date") >= start_date) & (F.col("date") <= end_date))
        stock2 = stock2.filter((F.col("date") >= start_date) & (F.col("date") <= end_date))

    joined_data = stock1.join(stock2, "date", "inner")

    correlation = joined_data.corr("close1", "close2")

    return correlation


In [23]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

def get_trading_value(dataset):

  df = dataset.withColumn("trading_value", col("Close") * col("Volume"))
  return df

In [24]:
import plotly.express as px
import plotly.graph_objects as go

def plot_stock():
  fig = go.Figure()
  fig.update_layout(title='Stocks over timme span', xaxis_title='Date', yaxis_title='Close')

  return fig

In [25]:
def add_trace_plot(dataset, fig):
  df_pandas = dataset.toPandas()
  fig.add_trace(go.Scatter(x=df_pandas['Date'], y=df_pandas['trading_value'], mode='lines', name='Other Value'))
  return fig

***

## Exploration

Loading all our datasets

In [28]:
folder = "drive/MyDrive/Spark/stocks_data/"

file1 = "AMAZON.csv"
file2 = "APPLE.csv"
file3 = "ZOOM.csv"
file4 = "TESLA.csv"
file5 = "FACEBOOK.csv"
file6 = "GOOGLE.csv"
file7 = "MICROSOFT.csv"

dataset = read_file(folder, file1)
dataset2 = read_file(folder, file2)
dataset3 = read_file(folder, file3)
dataset4 = read_file(folder, file4)
dataset5 = read_file(folder, file5)
dataset6 = read_file(folder, file6)
dataset7 = read_file(folder, file7)

In [29]:
get_info_df(dataset3)

Infos of the AMAZON.csv file 


► Schema of the dataset
root
 |-- Date: string (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

None

 ► Top rows of the dataset
+----------+-----------------+------------------+-----------------+-----------------+--------+-----------------+------------+
|      Date|             High|               Low|             Open|            Close|  Volume|        Adj Close|company_name|
+----------+-----------------+------------------+-----------------+-----------------+--------+-----------------+------------+
|2019-04-18|             66.0| 60.32099914550781|             65.0|             62.0|25764700|             62.0|        ZOOM|
|2019-04-22| 68.9000015258789|59.939998626708984|             61.0|65.69999694824219| 9949700|65.

In [30]:
get_info_df(dataset)

Infos of the AMAZON.csv file 


► Schema of the dataset
root
 |-- Date: string (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- company_name: string (nullable = true)

None

 ► Top rows of the dataset
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500|757.179992675

Average for opening and closing by year, month and week

**Year**

In [31]:
col1 ="Open"
col2 ="Close"

open_year = get_average_year(dataset, col1)
close_year = get_average_year(dataset, col2)

open_year.show(10)
close_year.show(10)

+----+------------------+
|year|      average_Open|
+----+------------------+
|2017|  968.275618959708|
|2018|1644.0727091633466|
|2019|1788.7461896623884|
|2020|2636.5054538710433|
+----+------------------+

+----+------------------+
|year|     average_Close|
+----+------------------+
|2017| 968.1670116409363|
|2018|1641.7261758629545|
|2019| 1789.189206077939|
|2020| 2636.649604240712|
+----+------------------+



**Month**

In [32]:
col1 ="Open"
col2 ="Close"

open_month = get_average_month(dataset, col1)
close_month = get_average_month(dataset, col2)

open_month.show(10)
close_month.show(10)

+----+-----+------------------+
|year|month|      average_Open|
+----+-----+------------------+
|2017|    1|  804.972998046875|
|2017|    2|  833.583162006579|
|2017|    3| 853.0417400857676|
|2017|    4| 904.8905286287006|
|2017|    5|  960.274092240767|
|2017|    6| 995.0422723943537|
|2017|    7|1007.6545013427734|
|2017|    8|  973.247826617697|
|2017|    9| 971.9529998779296|
|2017|   10| 999.5400057705966|
+----+-----+------------------+
only showing top 10 rows

+----+-----+------------------+
|year|month|     average_Close|
+----+-----+------------------+
|2017|    1| 807.5050018310546|
|2017|    2| 835.7452681692023|
|2017|    3| 854.2400008491848|
|2017|    4| 903.3857903731497|
|2017|    5| 961.7195462313565|
|2017|    6| 990.4440890225497|
|2017|    7|1008.4839965820313|
|2017|    8|    971.4365234375|
|2017|    9| 968.9924987792969|
|2017|   10|1000.7199956720525|
+----+-----+------------------+
only showing top 10 rows



**Week**

In [33]:
col1 ="Open"
col2 ="Close"

open_week = get_average_week(dataset, col1)
close_week = get_average_week(dataset, col2)

open_week.show(10)
close_week.show(10)

+----+----+-----------------+
|year|week|     average_Open|
+----+----+-----------------+
|2017|   1|765.0549926757812|
|2017|   2|800.5779907226563|
|2017|   3|812.6200103759766|
|2017|   4|825.8239990234375|
|2017|   5|  825.85400390625|
|2017|   6| 815.443994140625|
|2017|   7|837.2920043945312|
|2017|   8|852.0125122070312|
|2017|   9|849.4320068359375|
|2017|  10|   849.3419921875|
+----+----+-----------------+
only showing top 10 rows

+----+----+-----------------+
|year|week|    average_Close|
+----+----+-----------------+
|2017|   1|771.8224945068359|
|2017|   2|804.5240112304688|
|2017|   3|808.6424865722656|
|2017|   4|830.3520141601563|
|2017|   5|827.2719970703125|
|2017|   6|817.7340087890625|
|2017|   7|    840.966015625|
|2017|   8|852.3699951171875|
|2017|   9|849.1099975585937|
|2017|  10|849.7180053710938|
+----+----+-----------------+
only showing top 10 rows



Daily and monthly stock variation

**Daily**

In [34]:
stock_diff = daily_difference(dataset)
stock_diff.show(10)

+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name| stock_difference|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|             null|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500|757.1799926757812|      AMAZON|   3.510009765625|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100|780.4500122070312|      AMAZON|   23.27001953125|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200| 795.989990234375|      AMAZON|15.53997802734375|
|2017-

**Monthly**

In [35]:
stock_diff_month = monthly_difference(dataset)
stock_diff_month.show(10)

+----------+------------------+-----------------+-----------------+-----------------+-------+-----------------+------------+------------------+
|      Date|              High|              Low|             Open|            Close| Volume|        Adj Close|company_name|  stock_difference|
+----------+------------------+-----------------+-----------------+-----------------+-------+-----------------+------------+------------------+
|2017-01-03|  758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|              null|
|2017-02-01|  833.780029296875|824.9400024414062|829.2100219726562|832.3499755859375|3850200|832.3499755859375|      AMAZON| 78.67999267578125|
|2017-03-01| 854.8300170898438| 849.010009765625|853.0499877929688|853.0800170898438|2760100|853.0800170898438|      AMAZON| 20.73004150390625|
|2017-04-03|  893.489990234375|885.4199829101562|            888.0| 891.510009765625|3422300| 891.510009765625|      AMAZON| 38.42999267

Calculating the daily return

In [36]:
updateddf = calculate_daily_return(dataset)
updateddf.show(10)

+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name|     Daily_return|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|2017-01-03| 758.760009765625|747.7000122070312|757.9199829101562|753.6699829101562|3521100|753.6699829101562|      AMAZON|            -4.25|
|2017-01-04|759.6799926757812|754.2000122070312|758.3900146484375|757.1799926757812|2510500|757.1799926757812|      AMAZON|-1.21002197265625|
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100|780.4500122070312|      AMAZON| 18.9000244140625|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200| 795.989990234375|      AMAZON| 13.6300048828125|
|2017-

Average daily return by period

In [37]:
column_name = "Daily_return"

daily_return_year = get_average_year(updateddf, column_name)
daily_return_year.show(10)

daily_return_month = get_average_month(updateddf, column_name)
daily_return_month.show(10)

daily_return_week = get_average_week(updateddf, column_name)
daily_return_week.show(10)

+----+--------------------+
|year|average_Daily_return|
+----+--------------------+
|2017|-0.10860731877178785|
|2018| -2.3465333003921813|
|2019| 0.44301641555059523|
|2020|  0.1441503696687232|
+----+--------------------+

+----+-----+--------------------+
|year|month|average_Daily_return|
+----+-----+--------------------+
|2017|    1|  2.5320037841796874|
|2017|    2|  2.1621061626233553|
|2017|    3|  1.1982607634171196|
|2017|    4|  -1.504738255550987|
|2017|    5|  1.4454539905894885|
|2017|    6| -4.5981833718039775|
|2017|    7|  0.8294952392578125|
|2017|    8|  -1.811303180197011|
|2017|    9| -2.9605010986328124|
|2017|   10|  1.1799899014559658|
+----+-----+--------------------+
only showing top 10 rows

+----+----+--------------------+
|year|week|average_Daily_return|
+----+----+--------------------+
|2017|   1|  6.7675018310546875|
|2017|   2|     3.9460205078125|
|2017|   3| -3.9775238037109375|
|2017|   4|    4.52801513671875|
|2017|   5|     1.4179931640625|
|2017|   

Get the highest daily return either globaly or for a given period

In [38]:
start = "2017-01-03"
end = "2017-01-17"
sorteddf = get_highest_dr(updateddf, start, end)
sorteddf.show(10)

+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|      Date|             High|              Low|             Open|            Close| Volume|        Adj Close|company_name|     Daily_return|
+----------+-----------------+-----------------+-----------------+-----------------+-------+-----------------+------------+-----------------+
|2017-01-05|782.4000244140625| 760.260009765625|761.5499877929688|780.4500122070312|5830100|780.4500122070312|      AMAZON| 18.9000244140625|
|2017-01-06|799.4400024414062|  778.47998046875|782.3599853515625| 795.989990234375|5986200| 795.989990234375|      AMAZON| 13.6300048828125|
|2017-01-12|814.1300048828125|            799.5|800.3099975585938|813.6400146484375|4873900|813.6400146484375|      AMAZON|13.33001708984375|
|2017-01-11|            799.5| 789.510009765625|793.6599731445312|  799.02001953125|2992800|  799.02001953125|      AMAZON| 5.36004638671875|
|2017-

**Moving Average**

In [39]:
column = "Close"
n = 5
reference_date="2019-05-03"

moving = calculate_moving_average(dataset,column,n,reference_date)
moving.show(10)

+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+------------------+
|      Date|              High|               Low|              Open|             Close| Volume|         Adj Close|company_name|  Close_moving_avg|
+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+------------------+
|2019-05-03|1964.4000244140625|            1936.0|            1949.0|   1962.4599609375|6381600|   1962.4599609375|      AMAZON|           1927.95|
|2019-05-06|            1959.0|            1910.5|  1917.97998046875| 1950.550048828125|5417800| 1950.550048828125|      AMAZON|1930.3739990234376|
|2019-05-07|1949.0999755859375|1903.3800048828125| 1939.989990234375|            1921.0|5902100|            1921.0|      AMAZON|1929.2699951171876|
|2019-05-08|1935.3699951171875|            1910.0|1918.8699951171875|  1917.77001953125|4078600|  1917.770019531

**Rate of Return (RoR)**

RoR one day to another

RoR on a given time period

In [40]:
start_date = '2018-05-01'
period = 90
ror_df = get_ror_period(dataset, start_date, period)

ror_df.orderBy(F.desc("return_rate")).show()

+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+-------------------+
|      date|              High|               Low|              Open|             Close| Volume|         Adj Close|company_name|        return_rate|
+----------+------------------+------------------+------------------+------------------+-------+------------------+------------+-------------------+
|2018-07-25|1863.8399658203125|1822.6400146484375| 1829.300048828125|1863.6099853515625|3738200|1863.6099853515625|      AMAZON| 0.1778152603550999|
|2018-07-17|  1851.68994140625|1797.3800048828125|  1811.56005859375|1843.9300537109375|5682900|1843.9300537109375|      AMAZON|0.16537739836076173|
|2018-07-18|1858.8800048828125|  1831.27001953125|            1848.0|1842.9200439453125|4861900|1842.9200439453125|      AMAZON| 0.1647390647370897|
|2018-07-24|            1840.0|1809.3800048828125| 1829.010009765625| 1829.239990234375|4278700| 1829.2399

**Correlation between stocks**

In [41]:
dataset2.show()

+----------+------------------+------------------+------------------+------------------+----------+------------------+------------+
|      Date|              High|               Low|              Open|             Close|    Volume|         Adj Close|company_name|
+----------+------------------+------------------+------------------+------------------+----------+------------------+------------+
|2017-01-03|29.082500457763672|28.690000534057617|28.950000762939453|29.037500381469727|1.151276E8|27.277639389038086|       APPLE|
|2017-01-04|29.127500534057617|           28.9375|28.962499618530273|  29.0049991607666| 8.44724E7|27.247108459472656|       APPLE|
|2017-01-05| 29.21500015258789|28.952499389648438|28.979999542236328| 29.15250015258789| 8.87744E7| 27.38566780090332|       APPLE|
|2017-01-06|29.540000915527344| 29.11750030517578| 29.19499969482422|29.477500915527344|1.270076E8| 27.69097137451172|       APPLE|
|2017-01-09|29.857500076293945|29.485000610351562|29.487499237060547|29.7474

In [42]:
start_date="2017-01-10"
end_date="2017-02-02"

correlation_value = calculate_stock_correlation(dataset, dataset2)

print(f"Correlation coefficient between stocks on the given span: {correlation_value}")


Correlation coefficient between stocks on the given span: 0.9292860084204524


***

In [43]:
dataset = get_trading_value(dataset)
dataset2 = get_trading_value(dataset2)
dataset3 = get_trading_value(dataset3)
dataset4 = get_trading_value(dataset4)

In [44]:
fig = plot_stock()

add_trace_plot(dataset, fig)
add_trace_plot(dataset2, fig)
add_trace_plot(dataset3, fig)
add_trace_plot(dataset4, fig)

In [45]:
import plotly.express as px
import plotly.graph_objects as go

pandas_df = dataset.toPandas()
pandas_df2 = dataset2.toPandas()

fig = go.Figure()

fig.add_trace(go.Scatter(x=pandas_df['Date'], y=pandas_df['trading_value'], mode='lines', name='Dataset 1'))

fig.add_trace(go.Scatter(x=pandas_df2['Date'], y=pandas_df2['trading_value'], mode='lines', name='Dataset 2'))

fig.update_layout(title='Time Series Plot', xaxis_title='Date', yaxis_title='Close')
fig.show()

***